In [49]:
# ---------------------------------
# Modeling & Evaluation - Fraud Detection
# ---------------------------------

import pandas as pd
import numpy as np
import joblib

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, f1_score, average_precision_score
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

# -------------------------
# Load Data
# -------------------------
df = pd.read_csv("../data/processed/fraud_data_processed.csv")

# -------------------------
# Preprocess Columns
# -------------------------
# 1. Handle timestamps
if 'transaction_date' in df.columns:
    df['transaction_date'] = pd.to_datetime(df['transaction_date'])
    df['hour'] = df['transaction_date'].dt.hour
    df['day'] = df['transaction_date'].dt.day
    df['weekday'] = df['transaction_date'].dt.weekday
    df = df.drop(columns=['transaction_date'])

# 2. Drop ID-like columns (not useful for ML)
for col in ['user_id', 'transaction_id']:
    if col in df.columns:
        df = df.drop(columns=[col])

# 3. Handle missing values
df = df.fillna(df.median(numeric_only=True))

# 4. Separate target
X = df.drop(columns=['class'])
y = df['class']

# -------------------------
# Encode categorical variables
# -------------------------
cat_cols = X.select_dtypes(include=['object', 'category']).columns
for col in cat_cols:
    freq = X[col].value_counts(normalize=True)
    X[col] = X[col].map(freq)

# -------------------------
# Train-Test Split
# -------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# -------------------------
# Feature Scaling
# -------------------------
scaler = StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns, index=X_train.index)
X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

# -------------------------
# Handle Class Imbalance with SMOTE
# -------------------------
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

print("Resampled training set shape:", X_train_res.shape)
print("Class distribution after SMOTE:\n", pd.Series(y_train_res).value_counts())

# -------------------------
# Train Models
# -------------------------
# Logistic Regression
lr = LogisticRegression(max_iter=1000, class_weight='balanced', random_state=42)
lr.fit(X_train_res, y_train_res)
y_pred_lr = lr.predict(X_test)
y_prob_lr = lr.predict_proba(X_test)[:, 1]

print("=== Logistic Regression ===")
print(classification_report(y_test, y_pred_lr))
print("PR-AUC:", average_precision_score(y_test, y_prob_lr))

# Random Forest
rf = RandomForestClassifier(n_estimators=200, max_depth=12, random_state=42, n_jobs=-1)
rf.fit(X_train_res, y_train_res)
y_pred_rf = rf.predict(X_test)
y_prob_rf = rf.predict_proba(X_test)[:, 1]

print("=== Random Forest ===")
print(classification_report(y_test, y_pred_rf))
print("PR-AUC:", average_precision_score(y_test, y_prob_rf))

# -------------------------
# Compare Models
# -------------------------
results = pd.DataFrame({
    "Model": ["Logistic Regression", "Random Forest"],
    "F1 Score": [
        f1_score(y_test, y_pred_lr),
        f1_score(y_test, y_pred_rf)
    ],
    "PR-AUC": [
        average_precision_score(y_test, y_prob_lr),
        average_precision_score(y_test, y_prob_rf)
    ]
})

print("=== Summary ===")
print(results)

# -------------------------
# Stratified K-Fold Cross-Validation
# -------------------------
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
pr_auc_scores = []

for train_idx, val_idx in skf.split(X, y):
    X_tr, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_tr, y_val = y.iloc[train_idx], y.iloc[val_idx]

    X_tr_res, y_tr_res = smote.fit_resample(X_tr, y_tr)
    X_val = pd.DataFrame(scaler.transform(X_val), columns=X_val.columns, index=X_val.index)

    rf.fit(X_tr_res, y_tr_res)
    y_val_prob = rf.predict_proba(X_val)[:, 1]
    pr_auc_scores.append(average_precision_score(y_val, y_val_prob))

print("Cross-Validation PR-AUC: Mean =", np.mean(pr_auc_scores), ", Std =", np.std(pr_auc_scores))

# -------------------------
# Save Models
# -------------------------
joblib.dump(lr, "../models/logistic_regression.pkl")
joblib.dump(rf, "../models/random_forest.pkl")


Resampled training set shape: (219136, 201)
Class distribution after SMOTE:
 class
0    109568
1    109568
Name: count, dtype: int64
=== Logistic Regression ===
              precision    recall  f1-score   support

           0       0.97      0.94      0.95     27393
           1       0.54      0.69      0.60      2830

    accuracy                           0.92     30223
   macro avg       0.75      0.81      0.78     30223
weighted avg       0.93      0.92      0.92     30223

PR-AUC: 0.6599936793569618
=== Random Forest ===
              precision    recall  f1-score   support

           0       0.97      0.94      0.95     27393
           1       0.53      0.69      0.60      2830

    accuracy                           0.91     30223
   macro avg       0.75      0.81      0.78     30223
weighted avg       0.93      0.91      0.92     30223

PR-AUC: 0.701714544568999
=== Summary ===
                 Model  F1 Score    PR-AUC
0  Logistic Regression  0.603486  0.659994
1       

['../models/random_forest.pkl']